In [1]:
import regex as re
import pandas as pd
import numpy as np
import networkx as nx

## Load Arena 55K Pairwise Data

In [2]:
chatbot_df = pd.read_csv("hf://datasets/lmsys/lmsys-arena-human-preference-55k/train.csv", index_col=0)

In [3]:
chatbot_filtered = chatbot_df[chatbot_df.winner_tie == 0].drop(columns='winner_tie') # remove ties
#chatbot_filtered = chatbot_filtered[~chatbot_filtered.prompt.str.startswith('["System')] # remove special tasks?
chatbot_filtered['prompt'] = chatbot_filtered.prompt.str[2:-2] # cleanup prompt
chatbot_filtered['response_a'] = chatbot_filtered.response_a.str[2:-2] # cleanup response A
chatbot_filtered['response_b'] = chatbot_filtered.response_b.str[2:-2] # cleanup response A

In [4]:
chatbot_filtered['winner'] = chatbot_filtered.apply(lambda row: row['model_a'] if row['winner_model_a'] else row['model_b'], axis=1)
chatbot_filtered['loser'] = chatbot_filtered.apply(lambda row: row['model_a'] if row['winner_model_b'] else row['model_b'], axis=1)
chatbot_filtered

,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner,loser
id,,,,,,,,,
30192,gpt-4-1106-preview,gpt-4-0613,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",1,0,gpt-4-1106-preview,gpt-4-0613
53567,koala-13b,gpt-4-0613,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,0,1,gpt-4-0613,koala-13b
96401,llama-2-13b-chat,mistral-7b-instruct,How can I create a test set for a very rare ca...,Creating a test set for a very rare category c...,When building a classifier for a very rare cat...,1,0,llama-2-13b-chat,mistral-7b-instruct
198779,koala-13b,gpt-3.5-turbo-0314,What is the best way to travel from Tel-Aviv t...,The best way to travel from Tel Aviv to Jerusa...,The best way to travel from Tel-Aviv to Jerusa...,0,1,gpt-3.5-turbo-0314,koala-13b
292873,vicuna-13b,gpt-4-0314,"Construct a rap battle, in the style of Epic R...","[Zeus]\nYo, it's the king of the gods on the m...","(Verse 1 - Zeus)\n\nI'm the king of the gods, ...",0,1,gpt-4-0314,vicuna-13b
...,...,...,...,...,...,...,...,...,...
4294656694,gpt-4-0613,claude-1,"A simple mnemonic for \u03c0:\n\""How I wish I ...","Sure, let's break it down:\n\n1. \""How\"" has 3...",Here is how that mnemonic represents the digit...,1,0,gpt-4-0613,claude-1
4294692063,claude-2.0,llama-2-13b-chat,"In python, implement a naive Bayes with gaussi...",Here is an implementation of a naive Bayes cla...,Sure! Here's an implementation of a naive Baye...,1,0,claude-2.0,llama-2-13b-chat
4294710549,claude-1,alpaca-13b,"is it unethical to work on building weapons?"",...",Working on weapons technology raises some ethi...,It depends on the context. Weapons can be used...,1,0,claude-1,alpaca-13b


In [5]:
wins_df = chatbot_filtered[['winner', 'loser']].copy()
wins_df['wins'] = 1
wins_df = wins_df.groupby(by=['winner', 'loser'], sort=True, as_index=False).count()
wins_df = wins_df.pivot(index='winner', columns='loser', values='wins').replace({np.NaN: 0})
wins_df.to_csv('../fairpair/data/chatbot_arena/comparisons_cleaned.csv')
wins_df

loser,RWKV-4-Raven-14B,alpaca-13b,chatglm-6b,chatglm2-6b,chatglm3-6b,claude-1,claude-2.0,claude-2.1,claude-instant-1,codellama-34b-instruct,...,stripedhyena-nous-7b,tulu-2-dpo-70b,vicuna-13b,vicuna-33b,vicuna-7b,wizardlm-13b,wizardlm-70b,yi-34b-chat,zephyr-7b-alpha,zephyr-7b-beta
winner,,,,,,,,,,,,,,,,,,,,,
RWKV-4-Raven-14B,0.0,24.0,21.0,0.0,0.0,14.0,2.0,0.0,2.0,0.0,...,0.0,0.0,21.0,2.0,10.0,6.0,0.0,0.0,0.0,0.0
alpaca-13b,23.0,0.0,38.0,0.0,0.0,8.0,1.0,0.0,6.0,0.0,...,0.0,0.0,21.0,0.0,12.0,9.0,0.0,0.0,0.0,0.0
chatglm-6b,20.0,26.0,0.0,0.0,0.0,5.0,3.0,0.0,1.0,0.0,...,0.0,0.0,13.0,2.0,6.0,1.0,0.0,0.0,0.0,0.0
chatglm2-6b,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,1.0,9.0,...,0.0,0.0,2.0,4.0,0.0,1.0,4.0,0.0,4.0,8.0
chatglm3-6b,0.0,0.0,0.0,14.0,0.0,5.0,5.0,5.0,5.0,4.0,...,1.0,5.0,1.0,3.0,0.0,3.0,2.0,12.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wizardlm-13b,21.0,17.0,8.0,15.0,2.0,9.0,16.0,1.0,19.0,20.0,...,0.0,2.0,27.0,22.0,24.0,0.0,17.0,1.0,7.0,59.0
wizardlm-70b,0.0,0.0,0.0,23.0,3.0,10.0,25.0,20.0,52.0,22.0,...,3.0,7.0,27.0,56.0,17.0,16.0,0.0,12.0,3.0,3.0
yi-34b-chat,0.0,0.0,0.0,2.0,65.0,6.0,6.0,31.0,58.0,6.0,...,3.0,14.0,7.0,56.0,0.0,2.0,8.0,0.0,0.0,8.0


In [6]:
response_df = chatbot_filtered.melt(id_vars=['model_a', 'model_b'], value_vars=['response_a', 'response_b'], value_name='response')
response_df['model'] = response_df.apply(lambda row: row['model_a'] if row['variable'] == 'response_a' else row['model_b'], axis=1)
response_df = response_df[['model', 'response']]
response_df

,model,response
0,gpt-4-1106-preview,The question of whether it is morally right to...
1,koala-13b,A marriage license is a legal document that al...
2,llama-2-13b-chat,Creating a test set for a very rare category c...
3,koala-13b,The best way to travel from Tel Aviv to Jerusa...
4,vicuna-13b,"[Zeus]\nYo, it's the king of the gods on the m..."
...,...,...
79427,claude-1,Here is how that mnemonic represents the digit...
79428,llama-2-13b-chat,Sure! Here's an implementation of a naive Baye...
79429,alpaca-13b,It depends on the context. Weapons can be used...
79430,tulu-2-dpo-70b,"As an AI language model, I do not promote or c..."


In [7]:
length_df = response_df.copy()
length_df['response_length'] = length_df.response.str.len()
length_df = length_df[['model', 'response_length']].groupby('model', as_index=False).mean()
length_df['long_response'] = (length_df.response_length >= length_df.response_length.median()).astype(int)
length_df

,model,response_length,long_response
0,RWKV-4-Raven-14B,958.310388,0
1,alpaca-13b,385.409369,0
2,chatglm-6b,1074.943099,0
3,chatglm2-6b,1159.600509,0
4,chatglm3-6b,1185.141007,0
...,...,...,...
59,wizardlm-13b,1318.800000,0
60,wizardlm-70b,1543.451872,1
61,yi-34b-chat,2074.278520,1
62,zephyr-7b-alpha,1413.000000,1


In [8]:
comparison_df = chatbot_filtered[['model_a', 'model_b']].copy()
comparison_df = pd.DataFrame({'model': pd.concat([comparison_df['model_a'], comparison_df['model_b']]).astype(str)})
comparison_df['comparisons'] = 1
comparison_df = comparison_df.groupby('model', as_index=False).count().sort_values('comparisons')
comparison_df['often_compared'] = (comparison_df.comparisons >= comparison_df.comparisons.median()).astype(int)
comparison_df

,model,comparisons,often_compared
34,mistral-7b-instruct-v0.2,72,0
48,qwen1.5-4b-chat,124,0
50,qwen1.5-7b-chat,140,0
42,openchat-3.5-0106,159,0
13,falcon-180b-chat,203,0
...,...,...,...
22,gpt-4-0314,2923,1
7,claude-2.1,3969,1
23,gpt-4-0613,4306,1
19,gpt-3.5-turbo-0613,4866,1


In [9]:
firstModel_df = chatbot_filtered[['model_a']].copy()
firstModel_df['first_comparisons'] = 1
firstModel_df = firstModel_df.groupby('model_a', as_index=False).count().sort_values('first_comparisons').rename(columns={'model_a': 'model'})
firstModel_df = firstModel_df.merge(comparison_df[['model', 'comparisons']], on='model')
firstModel_df['first_percent'] = firstModel_df.first_comparisons / firstModel_df.comparisons # normalize by total #comparisons
firstModel_df['often_first'] = (firstModel_df.first_percent >= firstModel_df.first_percent.median()).astype(int)
firstModel_df.sort_values('first_percent')

,model,first_comparisons,comparisons,first_percent,often_first
3,openchat-3.5-0106,74,159,0.465409,0
45,llama-2-7b-chat,553,1174,0.471039,0
7,zephyr-7b-alpha,127,267,0.475655,0
21,mpt-7b-chat,307,643,0.477449,0
32,yi-34b-chat,465,973,0.477903,0
...,...,...,...,...,...
14,llama2-70b-steerlm-chat,242,458,0.528384,1
2,qwen1.5-7b-chat,74,140,0.528571,1
8,gpt4all-13b-snoozy,147,277,0.530686,1
17,deepseek-llm-67b-chat,283,519,0.545279,1


In [10]:
formatted_df = response_df.copy()
# detect formatting with (i) double newlines and (ii) double underscores or asterisks or ``` (markdown)
formatted_df['double_newlines'] = formatted_df.response.str.contains('\\n\\n', regex=False)
formatted_df['markdown'] = formatted_df.response.str.contains(r'\s(__|\*\*|```)(?!\s)(.(?!\1))+(?!\s(?=\1))', regex=True)
formatted_df['formatted'] = (formatted_df.double_newlines | formatted_df.markdown).astype(int)
formatted_df = formatted_df[['model', 'formatted']].groupby('model', as_index=False).sum()
formatted_df = formatted_df.merge(comparison_df[['model', 'comparisons']], on='model')
formatted_df['formatted_percent'] = formatted_df.formatted / formatted_df.comparisons # normalize by total #comparisons
formatted_df['often_formatted'] = (formatted_df.formatted_percent >= formatted_df.formatted_percent.median()).astype(int)
formatted_df.sort_values('often_formatted')

/tmp/ipykernel_703934/3883911278.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  formatted_df['markdown'] = formatted_df.response.str.contains(r'\s(__|\*\*|```)(?!\s)(.(?!\1))+(?!\s(?=\1))', regex=True)


,model,formatted,comparisons,formatted_percent,often_formatted
0,RWKV-4-Raven-14B,7,799,0.008761,0
25,gpt4all-13b-snoozy,124,277,0.447653,0
27,koala-13b,674,1102,0.611615,0
28,llama-13b,12,392,0.030612,0
62,zephyr-7b-alpha,161,267,0.602996,0
...,...,...,...,...,...
9,codellama-34b-instruct,814,1027,0.792600,1
21,gpt-4-0125-preview,656,798,0.822055,1
8,claude-instant-1,2309,2922,0.790212,1
45,pplx-70b-online,823,1015,0.810837,1


In [11]:
formatted_df.model.to_list()

['RWKV-4-Raven-14B',
 'alpaca-13b',
 'chatglm-6b',
 'chatglm2-6b',
 'chatglm3-6b',
 'claude-1',
 'claude-2.0',
 'claude-2.1',
 'claude-instant-1',
 'codellama-34b-instruct',
 'deepseek-llm-67b-chat',
 'dolly-v2-12b',
 'dolphin-2.2.1-mistral-7b',
 'falcon-180b-chat',
 'fastchat-t5-3b',
 'gemini-pro',
 'gemini-pro-dev-api',
 'gpt-3.5-turbo-0125',
 'gpt-3.5-turbo-0314',
 'gpt-3.5-turbo-0613',
 'gpt-3.5-turbo-1106',
 'gpt-4-0125-preview',
 'gpt-4-0314',
 'gpt-4-0613',
 'gpt-4-1106-preview',
 'gpt4all-13b-snoozy',
 'guanaco-33b',
 'koala-13b',
 'llama-13b',
 'llama-2-13b-chat',
 'llama-2-70b-chat',
 'llama-2-7b-chat',
 'llama2-70b-steerlm-chat',
 'mistral-7b-instruct',
 'mistral-7b-instruct-v0.2',
 'mistral-medium',
 'mixtral-8x7b-instruct-v0.1',
 'mpt-30b-chat',
 'mpt-7b-chat',
 'nous-hermes-2-mixtral-8x7b-dpo',
 'oasst-pythia-12b',
 'openchat-3.5',
 'openchat-3.5-0106',
 'openhermes-2.5-mistral-7b',
 'palm-2',
 'pplx-70b-online',
 'pplx-7b-online',
 'qwen-14b-chat',
 'qwen1.5-4b-chat',
 '

In [12]:
# OpenSource operationalized as available on HuggingFace
opensource_df = pd.DataFrame([{'model': 'RWKV-4-Raven-14B', 'open_source': True},
                              {'model': 'alpaca-13b', 'open_source': False},
                              {'model': 'chatglm-6b', 'open_source': True},
                              {'model': 'chatglm2-6b', 'open_source': True},
                              {'model': 'chatglm3-6b', 'open_source': True},
                              {'model': 'claude-1', 'open_source': False},
                              {'model': 'claude-2.0', 'open_source': False},
                              {'model': 'claude-2.1', 'open_source': False},
                              {'model': 'claude-instant-1', 'open_source': False},
                              {'model': 'codellama-34b-instruct', 'open_source': True},
                              {'model': 'deepseek-llm-67b-chat', 'open_source': True},
                              {'model': 'dolly-v2-12b', 'open_source': True},
                              {'model': 'dolphin-2.2.1-mistral-7b', 'open_source': True},
                              {'model': 'falcon-180b-chat', 'open_source': True},
                              {'model': 'fastchat-t5-3b', 'open_source': True},
                              {'model': 'gemini-pro', 'open_source': False},
                              {'model': 'gemini-pro-dev-api', 'open_source': False},
                              {'model': 'gpt-3.5-turbo-0125', 'open_source': False},
                              {'model': 'gpt-3.5-turbo-0314', 'open_source': False},
                              {'model': 'gpt-3.5-turbo-0613', 'open_source': False},
                              {'model': 'gpt-3.5-turbo-1106', 'open_source': False},
                              {'model': 'gpt-4-0125-preview', 'open_source': False},
                              {'model': 'gpt-4-0314', 'open_source': False},
                              {'model': 'gpt-4-0613', 'open_source': False},
                              {'model': 'gpt-4-1106-preview', 'open_source': False},
                              {'model': 'gpt4all-13b-snoozy', 'open_source': True},
                              {'model': 'guanaco-33b', 'open_source': False},
                              {'model': 'koala-13b', 'open_source': False},
                              {'model': 'llama-13b', 'open_source': True},
                              {'model': 'llama-2-13b-chat', 'open_source': True},
                              {'model': 'llama-2-70b-chat', 'open_source': True},
                              {'model': 'llama-2-7b-chat', 'open_source': True},
                              {'model': 'llama2-70b-steerlm-chat', 'open_source': True},
                              {'model': 'mistral-7b-instruct', 'open_source': True},
                              {'model': 'mistral-7b-instruct-v0.2', 'open_source': True},
                              {'model': 'mistral-medium', 'open_source': False},
                              {'model': 'mixtral-8x7b-instruct-v0.1', 'open_source': True},
                              {'model': 'mpt-30b-chat', 'open_source': True},
                              {'model': 'mpt-7b-chat', 'open_source': True},
                              {'model': 'nous-hermes-2-mixtral-8x7b-dpo', 'open_source': True},
                              {'model': 'oasst-pythia-12b', 'open_source': True},
                              {'model': 'openchat-3.5', 'open_source': True},
                              {'model': 'openchat-3.5-0106', 'open_source': True},
                              {'model': 'openhermes-2.5-mistral-7b', 'open_source': True},
                              {'model': 'palm-2', 'open_source': False},
                              {'model': 'pplx-70b-online', 'open_source': False},
                              {'model': 'pplx-7b-online', 'open_source': False},
                              {'model': 'qwen-14b-chat', 'open_source': True},
                              {'model': 'qwen1.5-4b-chat', 'open_source': True},
                              {'model': 'qwen1.5-72b-chat', 'open_source': True},
                              {'model': 'qwen1.5-7b-chat', 'open_source': True},
                              {'model': 'solar-10.7b-instruct-v1.0', 'open_source': True},
                              {'model': 'stablelm-tuned-alpha-7b', 'open_source': True},
                              {'model': 'starling-lm-7b-alpha', 'open_source': True},
                              {'model': 'stripedhyena-nous-7b', 'open_source': True},
                              {'model': 'tulu-2-dpo-70b', 'open_source': True},
                              {'model': 'vicuna-13b', 'open_source': True},
                              {'model': 'vicuna-33b', 'open_source': True},
                              {'model': 'vicuna-7b', 'open_source': True},
                              {'model': 'wizardlm-13b', 'open_source': True},
                              {'model': 'wizardlm-70b', 'open_source': True},
                              {'model': 'yi-34b-chat', 'open_source': True},
                              {'model': 'zephyr-7b-alpha', 'open_source': True},
                              {'model': 'zephyr-7b-beta', 'open_source': True}])
opensource_df['open_source'] = (opensource_df.open_source == False).astype(int) # invert to encode the privileged group
opensource_df

,model,open_source
0,RWKV-4-Raven-14B,0
1,alpaca-13b,1
2,chatglm-6b,0
3,chatglm2-6b,0
4,chatglm3-6b,0
...,...,...
59,wizardlm-13b,0
60,wizardlm-70b,0
61,yi-34b-chat,0
62,zephyr-7b-alpha,0


In [13]:
groups_df = length_df.merge(comparison_df, on='model').merge(firstModel_df.drop(columns='comparisons'), on='model')
groups_df = groups_df.merge(formatted_df.drop(columns='comparisons'), on='model').merge(opensource_df, on='model')
groups_df.to_csv('../fairpair/data/chatbot_arena/all_models.csv', index=False)
groups_df

,model,response_length,long_response,comparisons,often_compared,first_comparisons,first_percent,often_first,formatted,formatted_percent,often_formatted,open_source
0,RWKV-4-Raven-14B,958.310388,0,799,0,415,0.519399,1,7,0.008761,0,0
1,alpaca-13b,385.409369,0,982,1,487,0.495927,0,65,0.066191,0,1
2,chatglm-6b,1074.943099,0,826,0,410,0.496368,0,524,0.634383,0,0
3,chatglm2-6b,1159.600509,0,393,0,196,0.498728,0,266,0.676845,1,0
4,chatglm3-6b,1185.141007,0,695,0,354,0.509353,1,446,0.641727,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
59,wizardlm-13b,1318.800000,0,1055,1,517,0.490047,0,735,0.696682,1,0
60,wizardlm-70b,1543.451872,1,1122,1,539,0.480392,0,881,0.785205,1,0
61,yi-34b-chat,2074.278520,1,973,1,465,0.477903,0,812,0.834532,1,0
62,zephyr-7b-alpha,1413.000000,1,267,0,127,0.475655,0,161,0.602996,0,0


In [14]:
groups_df = pd.read_csv('../fairpair/data/chatbot_arena/all_models.csv')

## Arena-Hard Leaderboard

In [28]:
scores_df = pd.read_csv('../arena-hard/leaderboard/arena_hard_leaderboard_20240924.csv')
#scores_df = pd.read_csv('../arena-hard/leaderboard/arena_hard_leaderboard_20240731.csv')

#_df = scores_df[['model', 'score']].merge(groups_df, on='model', how='right')

# recalculate top/bottom 50%
#_df['long_response'] = (_df.response_length > _df.response_length.median()).astype(int)
#_df['often_compared'] = (_df.comparisons > _df.comparisons.median()).astype(int)
#_df['often_first'] = (_df.first_percent > _df.first_percent.median()).astype(int)
#_df['often_formatted'] = (_df.formatted > _df.formatted.median()).astype(int)
#arena_hard_combined = _df

# keep all models included in the 55K dataset
arena_hard_combined = scores_df[['model', 'score']].merge(groups_df, on='model', how='right')

arena_hard_combined.sort_values('score', ascending=False).reset_index(drop=True)
arena_hard_combined.to_csv('../fairpair/data/chatbot_arena/arena_hard_fullMedian.csv', index=False)
arena_hard_combined

,model,score,response_length,long_response,comparisons,often_compared,first_comparisons,first_percent,often_first,formatted,formatted_percent,often_formatted,open_source
0,RWKV-4-Raven-14B,NaN,958.310388,0,799,0,415,0.519399,1,7,0.008761,0,0
1,alpaca-13b,NaN,385.409369,0,982,1,487,0.495927,0,65,0.066191,0,1
2,chatglm-6b,NaN,1074.943099,0,826,0,410,0.496368,0,524,0.634383,0,0
3,chatglm2-6b,NaN,1159.600509,0,393,0,196,0.498728,0,266,0.676845,1,0
4,chatglm3-6b,NaN,1185.141007,0,695,0,354,0.509353,1,446,0.641727,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,wizardlm-13b,NaN,1318.800000,0,1055,1,517,0.490047,0,735,0.696682,1,0
60,wizardlm-70b,NaN,1543.451872,1,1122,1,539,0.480392,0,881,0.785205,1,0
61,yi-34b-chat,23.15,2074.278520,1,973,1,465,0.477903,0,812,0.834532,1,0
62,zephyr-7b-alpha,NaN,1413.000000,1,267,0,127,0.475655,0,161,0.602996,0,0


In [16]:
comparison_hard_df = pd.read_json('../arena-hard/data/arena_hard_battles.jsonl', lines=True)
comparison_hard_df # always compared to gpt-4-0314 as a baseline, so more of a stargraph than truly pairwise

,question_id,model_a,model_b,winner
0,93c9b4bb32f4422f954c7af9ea2bdfcd,gpt-4-0314,llama-3.1-8b-instruct,model_a
1,93c9b4bb32f4422f954c7af9ea2bdfcd,gpt-4-0314,llama-3.1-8b-instruct,model_a
2,93c9b4bb32f4422f954c7af9ea2bdfcd,gpt-4-0314,llama-3.1-8b-instruct,model_a
3,93c9b4bb32f4422f954c7af9ea2bdfcd,gpt-4-0314,llama-3.1-8b-instruct,model_a
4,93c9b4bb32f4422f954c7af9ea2bdfcd,gpt-4-0314,llama-3.1-8b-instruct,model_a
...,...,...,...,...
88737,de6e5b0884554e3a80d7c29e72d9306a,gpt-4-0314,claude-2.0,model_a
88738,f9111d1c39744147976e90c820838582,gpt-4-0314,claude-2.0,model_a
88739,f9111d1c39744147976e90c820838582,gpt-4-0314,claude-2.0,model_b
88740,7956046cc15646909bd07c31d0ea0371,gpt-4-0314,claude-2.0,model_a


## AlpacaEval 2.0 Leaderboard

In [30]:
scoresAlpaca_df = pd.read_csv('../fairpair/data/chatbot_arena/weighted_alpaca_eval_gpt4_turbo_leaderboard.csv').rename(columns={'Unnamed: 0': 'model'})
#_df = scoresAlpaca_df[['model', 'win_rate']].merge(groups_df, on='model')

# recalculate top/bottom 50%
#_df['long_response'] = (_df.response_length > _df.response_length.median()).astype(int)
#_df['often_compared'] = (_df.comparisons > _df.comparisons.median()).astype(int)
#_df['often_first'] = (_df.first_percent > _df.first_percent.median()).astype(int)
#_df['often_formatted'] = (_df.formatted > _df.formatted.median()).astype(int)
#alpaca_combined = _df.rename(columns={'win_rate': 'score'})

# keep all models included in the 55K dataset
alpaca_combined = scoresAlpaca_df[['model', 'win_rate']].merge(groups_df, on='model', how='right').rename(columns={'win_rate': 'score'})

alpaca_combined.sort_values('score', ascending=False).reset_index(drop=True)
alpaca_combined.to_csv('../fairpair/data/chatbot_arena/alpaca_fullMedian.csv', index=False)
alpaca_combined

,model,score,response_length,long_response,comparisons,often_compared,first_comparisons,first_percent,often_first,formatted,formatted_percent,often_formatted,open_source
0,RWKV-4-Raven-14B,NaN,958.310388,0,799,0,415,0.519399,1,7,0.008761,0,0
1,alpaca-13b,NaN,385.409369,0,982,1,487,0.495927,0,65,0.066191,0,1
2,chatglm-6b,NaN,1074.943099,0,826,0,410,0.496368,0,524,0.634383,0,0
3,chatglm2-6b,2.762185,1159.600509,0,393,0,196,0.498728,0,266,0.676845,1,0
4,chatglm3-6b,NaN,1185.141007,0,695,0,354,0.509353,1,446,0.641727,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,wizardlm-13b,5.878153,1318.800000,0,1055,1,517,0.490047,0,735,0.696682,1,0
60,wizardlm-70b,14.383896,1543.451872,1,1122,1,539,0.480392,0,881,0.785205,1,0
61,yi-34b-chat,NaN,2074.278520,1,973,1,465,0.477903,0,812,0.834532,1,0
62,zephyr-7b-alpha,8.352664,1413.000000,1,267,0,127,0.475655,0,161,0.602996,0,0


## HELM Lite Leaderboard

In [18]:
helm_df = pd.DataFrame([
    {'model': 'claude-2.0', 'win_rate': 0.55},
    {'model': 'claude-2.1', 'win_rate': 0.494},
    {'model': 'claude-instant-1', 'win_rate': 0.455},
    {'model': 'deepseek-llm-67b-chat', 'win_rate': 0.546},
    {'model': 'gemini-pro', 'win_rate': 0.474},
    {'model': 'gpt-3.5-turbo-0613', 'win_rate': 0.408},
    {'model': 'gpt-4-0613', 'win_rate': 0.915},
    {'model': 'gpt-4-1106-preview', 'win_rate': 0.756},
    {'model': 'llama-2-13b-chat', 'win_rate': 0.265},
    {'model': 'llama-2-7b-chat', 'win_rate': 0.174},
    {'model': 'mistral-7b-instruct', 'win_rate': 0.329},
    {'model': 'mistral-medium', 'win_rate': 0.309},
    {'model': 'mixtral-8x7b-instruct-v0.1', 'win_rate': 0.571},
    {'model': 'palm-2', 'win_rate': 0.693},
    {'model': 'qwen1.5-72b-chat', 'win_rate': 0.671},
    {'model': 'yi-34b-chat', 'win_rate': 0.626},
])

In [33]:
_df = helm_df[['model', 'win_rate']].merge(groups_df, on='model')

# recalculate top/bottom 50%
_df['long_response'] = (_df.response_length > _df.response_length.median()).astype(int)
_df['often_compared'] = (_df.comparisons > _df.comparisons.median()).astype(int)
_df['often_first'] = (_df.first_percent > _df.first_percent.median()).astype(int)
_df['often_formatted'] = (_df.formatted > _df.formatted.median()).astype(int)
helm_combined = _df.rename(columns={'win_rate': 'score'})

# keep all models included in the 55K dataset
#helm_combined = helm_df[['model', 'win_rate']].merge(groups_df, on='model', how='right').rename(columns={'win_rate': 'score'})

helm_combined.sort_values('score', ascending=False).reset_index(drop=True)
helm_combined.to_csv('../fairpair/data/chatbot_arena/helm_combined.csv', index=False)
helm_combined

,model,score,response_length,long_response,comparisons,often_compared,first_comparisons,first_percent,often_first,formatted,formatted_percent,often_formatted,open_source
0,claude-2.0,0.550,1224.380241,0,1741,0,908,0.521539,1,1377,0.790925,0,1
1,claude-2.1,0.494,1247.284203,0,3969,1,2058,0.518519,1,3061,0.771227,1,1
2,claude-instant-1,0.455,1304.210472,0,2922,1,1455,0.497947,0,2309,0.790212,1,1
3,deepseek-llm-67b-chat,0.546,1336.406551,0,519,0,283,0.545279,1,369,0.710983,0,0
4,gemini-pro,0.474,1382.902778,0,1008,0,503,0.499008,0,702,0.696429,0,1
5,gpt-3.5-turbo-0613,0.408,1407.265105,1,4866,1,2448,0.503083,1,3043,0.625360,1,1
6,gpt-4-0613,0.915,1300.464933,0,4306,1,2167,0.503251,1,2723,0.632373,1,1
7,gpt-4-1106-preview,0.756,2295.297948,1,5360,1,2669,0.497948,0,4564,0.851493,1,1
8,llama-2-13b-chat,0.265,1706.378180,1,1769,1,875,0.494630,0,1553,0.877897,1,0
9,llama-2-7b-chat,0.174,1780.380750,1,1174,0,553,0.471039,0,898,0.764906,0,0


## Create a FairPairGraph

In [20]:
def load_dataset(ground_truth_file:str, comparisons_file:str, group_attribute:str) -> nx.DiGraph:
    
    wins_df = pd.read_csv(comparisons_file, index_col=0)
    benchmark_df = pd.read_csv(ground_truth_file, index_col=0) # model names as index

    G = nx.from_pandas_adjacency(wins_df, create_using=nx.DiGraph)
    G2 = G.subgraph(nodes=benchmark_df.index).copy()  # a smaller graph with only the models that we have benchmark data for

    attr_df = benchmark_df[['score', group_attribute]].rename(columns={'score': 'skill'})
    attr_df['unpriv'] = attr_df[group_attribute] == 0
    attr_df = attr_df[['skill', 'unpriv']].to_dict(orient='index')

    nx.set_node_attributes(G2, attr_df)

    return G2

In [21]:
G = load_dataset(ground_truth_file='../fairpair/data/chatbot_arena/helm_combined.csv',
                 comparisons_file='../fairpair/data/chatbot_arena/comparisons_cleaned.csv',
                 group_attribute='long_response')

In [22]:
G.nodes(data=True)

NodeDataView({'claude-instant-1': {'skill': 0.455, 'unpriv': True}, 'mistral-7b-instruct': {'skill': 0.329, 'unpriv': True}, 'claude-2.0': {'skill': 0.55, 'unpriv': True}, 'gpt-3.5-turbo-0613': {'skill': 0.408, 'unpriv': False}, 'gpt-4-0613': {'skill': 0.915, 'unpriv': True}, 'mistral-medium': {'skill': 0.309, 'unpriv': False}, 'gemini-pro': {'skill': 0.474, 'unpriv': True}, 'llama-2-13b-chat': {'skill': 0.265, 'unpriv': False}, 'mixtral-8x7b-instruct-v0.1': {'skill': 0.571, 'unpriv': False}, 'palm-2': {'skill': 0.693, 'unpriv': True}, 'claude-2.1': {'skill': 0.494, 'unpriv': True}, 'qwen1.5-72b-chat': {'skill': 0.671, 'unpriv': False}, 'yi-34b-chat': {'skill': 0.626, 'unpriv': False}, 'llama-2-7b-chat': {'skill': 0.174, 'unpriv': False}, 'gpt-4-1106-preview': {'skill': 0.756, 'unpriv': False}, 'deepseek-llm-67b-chat': {'skill': 0.546, 'unpriv': True}})

In [23]:
import random

[random.randint(1,100) for i in range(10)]

[73, 55, 39, 12, 24, 23, 70, 3, 49, 94]